In [ ]:
sys.argv.append('--JOB_NAME')
sys.argv.append('James1')

sys.argv.append('--perfect_match_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-landing-zone/data-and-insight/address-matching-glue-job-output/perfect_match_s3_bucket_target')

sys.argv.append('--best_match_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-landing-zone/data-and-insight/address-matching-glue-job-output/best_match_s3_bucket_target')

sys.argv.append('--non_match_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-landing-zone/data-and-insight/address-matching-glue-job-output/non_match_s3_bucket_target')

sys.argv.append('--imperfect_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-landing-zone/data-and-insight/address-matching-glue-job-output/imperfect_s3_bucket_target')

sys.argv.append('--query_addresses_url')
sys.argv.append('s3://dataplatform-stg-landing-zone/data-and-insight/address-matching-test/test_addresses.gz.parquet')

sys.argv.append('--target_addresses_url')
sys.argv.append('s3://dataplatform-stg-landing-zone/data-and-insight/address-matching-test/addresses_api_full.gz.parquet')

Starting Spark application


In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, NGram, HashingTF, MinHashLSH
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
import pyspark.sql.functions as F
from awsglue.dynamicframe import DynamicFrame

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default

## write into the log file with:
## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

# Environment Arguments
perfect_match_s3_bucket_target = get_glue_env_var('perfect_match_s3_bucket_target', '')
best_match_s3_bucket_target = get_glue_env_var('best_match_s3_bucket_target', '')
imperfect_s3_bucket_target = get_glue_env_var('imperfect_s3_bucket_target', '')
non_match_s3_bucket_target = get_glue_env_var('non_match_s3_bucket_target', '')

query_addresses_url = get_glue_env_var('query_addresses_url', '')
target_addresses_url = get_glue_env_var('target_addresses_url', '')

# Context Setup
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
logger = glueContext.get_logger()
job = Job(glueContext)

job.init('James1', args)

In [ ]:
#logger.info('fetch first set of data')
query_addresses = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    format="parquet",
    connection_options={
        "paths": [query_addresses_url],
        "recurse": True
    },
    transformation_ctx="query_addresses"
)
query_addresses_df = query_addresses.toDF()

In [ ]:
logger.info('concat first set of data')
query_concat = query_addresses_df.withColumn(
    "concat_address",
    F.concat_ws(" ", "concatenated_string_to_match", "postcode")
)

In [ ]:
logger.info('fetch second set of data')
target_addresses = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    format="parquet",
    connection_options={"paths": [target_addresses_url], "recurse": True},
    transformation_ctx="target_addresses"
)
target_addresses_df = target_addresses.toDF()

In [ ]:
logger.info('concat seconds set of data')
target_concat_address = target_addresses_df.withColumn(
    "concat_address",
    F.concat_ws(" ", "concatenated2", "postcode")
)

In [ ]:
# would be nice to make this optional: that creates a subset of LLPG based on the postcodes in the list of addresses to match
target_concat_address = target_concat_address.join(query_concat, "postcode","leftsemi")

model = Pipeline(stages=[
    RegexTokenizer(
        pattern="", inputCol="concat_address", outputCol="tokens", minTokenLength=1
    ),
    NGram(n=3, inputCol="tokens", outputCol="ngrams"),
    HashingTF(inputCol="ngrams", outputCol="vectors"),
    MinHashLSH(inputCol="vectors", outputCol="lsh")
]).fit(target_concat_address)

target_hashed = model.transform(target_concat_address)
query_hashed = model.transform(query_concat)

joined = model.stages[-1].approxSimilarityJoin(
    query_hashed.withColumnRenamed("concat_address", "query_address"),
    target_hashed.withColumnRenamed("concat_address", "result_address"),
    1 # when in the restricted mode (line 77), it's better to set the threshold to 0.7
)

window = Window.partitionBy(joined['datasetA.query_address']).orderBy(joined['distCol'])
ranked = joined.select('*', rank().over(window).alias('rank'))
ranked_frame = ranked.select(
    'datasetA.query_address',
    'datasetB.result_address',
    'datasetB.uprn',
    'distCol',
    'rank',
    'datasetA.prinx'
)

In [ ]:
# only keep best match and remove duplicates
bestMatch = ranked_frame.filter(col('rank') == 1).dropDuplicates(['prinx'])
nonMatch = query_concat.join(bestMatch, "prinx", "leftanti")
perfectMatch = bestMatch.filter('distCol = 0')
imperfectMatch = bestMatch.filter('distCol != 0')
# results = bestMatch.union(nonMatch)

In [ ]:
bestMatchframe = DynamicFrame.fromDF(bestMatch, glueContext, "bestMatch")
nonMatchframe = DynamicFrame.fromDF(nonMatch, glueContext, "nonMatch")
imperfectMatchframe = DynamicFrame.fromDF(imperfectMatch, glueContext, "imperfectMatch")
perfectMatchframe = DynamicFrame.fromDF(perfectMatch, glueContext, "perfectMatch")

In [ ]:
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=perfectMatch,
    connection_type="s3",
    format="parquet",
    connection_options={"path": perfect_match_s3_bucket_target, "partitionKeys": []},
    transformation_ctx="parquetData")

In [ ]:
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=bestMatchframe,
    connection_type="s3",
    format="parquet",
    connection_options={"path": best_match_s3_bucket_target, "partitionKeys": []},
    transformation_ctx="parquetData")

In [ ]:
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=imperfectMatch,
    connection_type="s3",
    format="parquet",
    connection_options={"path": imperfect_s3_bucket_target, "partitionKeys": []},
    transformation_ctx="parquetData")

In [ ]:
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=nonMatchframe,
    connection_type="s3",
    format="parquet",
    connection_options={"path": non_match_s3_bucket_target, "partitionKeys": []},
    transformation_ctx="parquetData")

In [ ]:
job.commit()